# Hito 8
# Implementación de buffer
Queremos conocer el número de chunks que pasan en el tiempo `jitter`, para esto, vemos cuántos bits se transportan en ese tiempo:

Sabemos que `bit_rate` (2901kbits/s) es el número de bits que se transportan en un segundo, entonces:

$-----------$

1s -------- 2901kbits

0.23s ----- ?

$-----------$



In [2]:
import math
bit_rate = 2901000 # en bits
jitter = 0.23      # En segundos
chunk_size = 1024 * 16  # En bits
bits_jitter = jitter * bit_rate
print("bits_jitter = ", bits_jitter, "bits")

bits_jitter =  667230.0 bits


Ahora, buscamos a cuantos chunks corresponde un `bits_jitter`

Para conseguir N:

N = $\frac{bits-jitter}{chunk-size}$

In [3]:
n = math.ceil(bits_jitter / chunk_size)
print("N =", n,"celdas")

N = 41 celdas


## Hilos - [threading](https://docs.python.org/es/3.8/library/threading.html#module-threading)
Tenemos 3 hilos:
![Hilos](https://github.com/RaquelGG/TM/blob/master/otros/Hilos.png?raw=true)

## Cerrojo - [threading.Lock()](https://docs.python.org/es/3.8/library/threading.html#lock-objects)
Se usa para la sincronización concurrente, en nuestro caso desbloquea el hilo playback siempre y cuando esté lleno al menos la mitad del buffer.


In [4]:
import threading
lock = threading.Lock() # Inicializa el cerrojo
lock.acquire() # Bloquea el cerrojo
lock.release() # Desbloquea el cerrojo

## Cola de prioridad - [PriorityQueue](https://docs.python.org/es/3.8/library/queue.html?highlight=priorityqueue#queue.PriorityQueue)
Las entradas de menor valor se obtienen primero:


![Imagen de una PriorityQueue](https://github.com/RaquelGG/TM/blob/master/otros/PriorityQueue.png?raw=true)

Cada elemento de la cola de prioridad contiene:
- Número de secuencia
- Trozo grabado

In [9]:
from queue import PriorityQueue
buffer = PriorityQueue(maxsize=n * 2) # Inicializa el buffer como cola de prioridad
buffer.put("id, chunk") # Añade un elemento a la cola
buffer.get() # Borra y devuelve el elemento más pequeño de la cola

'id, chunk'

## should_skip_chunk(seq)
Evita almacenar trozos con un número de secuencia algo menor al último número de secuencia reproducido.

En caso de ser la diferencia mayor que ```N```·2 se cambia el último número de secuencia reproducido por el de la secuencia recibida - 1

## Conclusión
Como el jitter se midió en una situación de sobrecarga de la red, el retardo que se produce con el ```N``` calculado es considerable, nosotros recomendamos usar un valor más bajo, como por ejemplo ```N``` = 10.

En las pruebas, incluso con ```N``` = 1 no había prácticamente pérdidas de paquetes ni desorden ni retardo que dificultara en ninguna medida la comunicación.